In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import random
from pprint import pprint

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
target_map = {i: name for i, name in enumerate(iris.target_names)}
df['target'] = df['target'].map(target_map)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   target        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [46]:
df.rename(columns={'sepal length (cm)':'sepal_length',
                   'sepal width (cm)':'sepal_width',
                   'petal length (cm)':'petal_length',
                   'petal width (cm)':'petal_width'},
                   inplace=True)

---
## Train Test Split function

In [3]:
def train_test_split(df, test_size, seed=None):
    seed = random.seed(seed)
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    idxs = df.index.tolist()
    test_idxs = random.sample(population=idxs, k=test_size)
    test_df = df.loc[test_idxs]
    train_df = df.drop(test_idxs)

    return train_df, test_df

---
## Helper Functions

### Is Data Pure?

In [4]:
def check_purity(data):
    if isinstance(data, pd.DataFrame):
        data = data.values
    target_col = data[:,-1]
    unique_classes = np.unique(target_col)
    if len(unique_classes) == 1:
        return True
    else:
        return False

### Classify

In [5]:
def classify_data(data):
    if isinstance(data, pd.DataFrame):
        data = data.values
    target_col = data[:,-1]
    unique_classes, class_counts = np.unique(target_col, return_counts=True)
    index = class_counts.argmax()
    classification = unique_classes[index]
    return classification

### Potential Splits

In [122]:
def get_potential_splts(data):
    
    if isinstance(data, pd.DataFrame):
        data = data.values
        
    potential_splits = {}
    n_cols = data.shape[1]

    for col_idx in range(n_cols -1):
        
        potential_splits[col_idx] = []
        vals = data[:, col_idx]
        unique_vals = np.unique(vals)

        potential_splits[col_idx] = unique_vals

    return potential_splits

### Split data

In [105]:
def split_data(data, split_col, split_val):

    if isinstance(data, pd.DataFrame):
        data = data.values

    split_col_vals = data[:, split_col]

    feature_type = FEATURE_TYPES[split_col]
    
    if feature_type == 'continuous':
        data_below = data[split_col_vals <= split_val]
        data_above = data[split_col_vals > split_val]
    else:
        data_below = data[split_col_vals == split_val]
        data_above = data[split_col_vals != split_val]

    return data_below, data_above

### Lowest Overall Entropy

In [8]:
def calculate_entropy(data):

    if isinstance(data, pd.DataFrame):
        data = data.values

    target_col = data[:,-1]
    counts = np.unique(target_col, return_counts=True)[1]

    probabilities = counts/counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

In [9]:
def calculate_overall_entropy(data_below, data_above):

    n_data = len(data_below) + len(data_above)
    
    p_data_below = len(data_below) / n_data
    p_data_above = len(data_above) / n_data

    overall_entropy = (p_data_below * calculate_entropy(data_below) +
                       p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [125]:
def determine_best_split(data, potential_splits):

    if isinstance(data, pd.DataFrame):
        data = data.values

    overall_entropy = 99999

    for col_idx in potential_splits:
        for val in potential_splits[col_idx]:
            data_below, data_above = split_data(data, split_col=col_idx, split_val=val)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_col = col_idx
                best_split_val = val

    return best_split_col, best_split_val

In [109]:
def get_feature_type(df):

    feature_types = []
    threshold = 15

    for col in df.columns:
        unique_vals = df[col].unique()
        example_val = unique_vals[0]
        if isinstance(example_val, str) or (len(unique_vals) <= threshold):
            feature_types.append('categorical')
        else:
            feature_types.append('continuous')

    return feature_types

---
## Decision Tree Algorithm

#### Representation of Decision Tree
```Python
sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

example_tree = {"petal_width <= 0.8": ["Iris-setosa", 
                                      {"petal_width <= 1.65": [{"petal_length <= 4.9": ["Iris-versicolor", 
                                                                                        "Iris-virginica"]}, 
                                                                "Iris-virginica"]}]}

```

In [126]:
def decision_tree_classifier(df, counter=0, min_samples=2, max_depth=5):

    # Data preparation
    if counter == 0:
        global COLUMN_NAMES, FEATURE_TYPES
        COLUMN_NAMES = df.columns
        FEATURE_TYPES = get_feature_type(df)
        data=df.values
    else:
        data = df

    # Base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    
    # Recursion
    else:
        counter += 1

        # Helper functions
        potential_splits = get_potential_splts(data)
        split_col, split_val = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_col, split_val)

        # Check for empty potential splits lists
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification

        # Instatiate sub tree
        feature_name = COLUMN_NAMES[split_col]
        feature_type = FEATURE_TYPES[split_col]
        
        if feature_type == 'continnuous':
            question = f'{feature_name} <= {split_val}'
        else:
            question = f'{feature_name} = {split_val}'

        sub_tree = {question: []}            

        # Find answers using recursion
        yes_answer = decision_tree_classifier(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_classifier(data_above, counter, min_samples, max_depth)

        # Check final answer and append question answers
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)

        return sub_tree

In [49]:
train, test = train_test_split(df, test_size=0.2, seed=42)

In [74]:
tree = decision_tree_classifier(train, max_depth=7)

In [71]:
pprint(tree)

{'petal_width <= 0.7': ['setosa',
                        {'petal_width <= 1.75': [{'petal_length <= 4.95': [{'petal_width <= 1.65': ['versicolor',
                                                                                                    'virginica']},
                                                                           {'petal_width <= 1.55': ['virginica',
                                                                                                    'versicolor']}]},
                                                 'virginica']}]}


---
## Classification

In [52]:
example = test.iloc[0]
example

sepal_length       5.2
sepal_width        3.4
petal_length       1.4
petal_width        0.2
target          setosa
Name: 28, dtype: object

In [116]:
def classify_example(example, tree):

    question = list(tree.keys())[0]
    feature_name, operator, value = question.split()

    # Ask question
    if operator == '<=':
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # Base case
    if not isinstance(answer, dict):
        return answer
    # Recursion
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [55]:
classify_example(example, tree)

'setosa'

---
## Accuracy

In [56]:
def calculate_accuracy(df, tree):

    df['classification'] = df.apply(classify_example, axis=1, args=(tree,))
    df['classification_correct'] = df.classification == df.target

    accuracy = df.classification_correct.mean()

    return accuracy

In [57]:
calculate_accuracy(test, tree)

0.9333333333333333

In [66]:
test[test['classification_correct']==False]

,sepal_length,sepal_width,petal_length,petal_width,target,classification,classification_correct
70,5.9,3.2,4.8,1.8,versicolor,virginica,False
129,7.2,3.0,5.8,1.6,virginica,versicolor,False


In [68]:
test.loc[70]

sepal_length                     5.9
sepal_width                      3.2
petal_length                     4.8
petal_width                      1.8
target                    versicolor
classification             virginica
classification_correct         False
Name: 70, dtype: object

In [69]:
pprint(tree)

{'petal_width <= 0.7': ['setosa',
                        {'petal_width <= 1.75': [{'petal_length <= 4.95': [{'petal_width <= 1.65': ['versicolor',
                                                                                                    'virginica']},
                                                                           {'petal_width <= 1.55': ['virginica',
                                                                                                    'versicolor']}]},
                                                 'virginica']}]}


In [72]:
train, test = train_test_split(df, test_size=0.2, seed=13)
tree = decision_tree_classifier(train, max_depth=3)
accuracy = calculate_accuracy(test, tree)

pprint(tree)
print(accuracy)

{'petal_width <= 0.8': ['setosa',
                        {'petal_width <= 1.75': [{'petal_length <= 4.95': ['versicolor',
                                                                           'virginica']},
                                                 'virginica']}]}
0.9666666666666667


---
## Handling Categoric Features

#### Titanic Dataset

In [99]:
# Load dataset from seaborn datasets
df = sns.load_dataset('titanic')

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [89]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [101]:
# Load dataset from seaborn datasets
df = sns.load_dataset('titanic')

# Reorganise to the dataset fits requirements of decision tree functions

# 1. must have target column called target as last column
df['target'] = df.survived

# Drop some unecessary columns for this example
df.drop(['survived','alive','deck','adult_male','class','embarked'], axis=1, inplace=True)

# 2. No missing values
median_age = df.age.median()
mode_embark = df.embark_town.mode()[0]

df.fillna({'age':median_age,
           'embark_town':mode_embark},
           inplace=True)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pclass       891 non-null    int64  
 1   sex          891 non-null    object 
 2   age          891 non-null    float64
 3   sibsp        891 non-null    int64  
 4   parch        891 non-null    int64  
 5   fare         891 non-null    float64
 6   who          891 non-null    object 
 7   embark_town  891 non-null    object 
 8   alone        891 non-null    bool   
 9   target       891 non-null    int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 63.6+ KB


In [127]:
train, test = train_test_split(df, test_size=0.2)
tree = decision_tree_classifier(train, max_depth=10)
accuracy = calculate_accuracy(test, tree)

pprint(tree)
print(accuracy)

{'who = man': [{'pclass = 1': [{'fare = 26.0': [0,
                                                {'fare = 30.5': [{'age = 52.0': [{'embark_town = Southampton': [{'age = 29.0': [{'age = 28.0': [1,
                                                                                                                                                0]},
                                                                                                                                1]},
                                                                                                                {'fare = 29.7': [{'fare = 26.55': [1,
                                                                                                                                                   {'fare = 27.7208': [0,
                                                                                                                                                                       {'alone = True': [1,
            